## MySQL
***


### Instalação

Rode o docker do mysql

```
pip3 install mysql-connector
```

### Entrar no MySQL

```
mysql -u <username> -p
senha: ****
exit
```

Se precisar colocar o host como localhost use o -h

### Backup

```
mysqldump -u <username> -p <database> > backup.sql
```


***
### Conexão
***

In [1]:
from mysql import connector

In [2]:
# Connection é o objeto que representa o banco de dados
conexao = connector.connect(
    host="mysql",
    user="root",
    passwd="docker",
    database="notebook_db"
)

In [3]:
# Vamos criar um cursor, ou seja, um iterador que permite navegar e manipular os registros do DB.
cursor = conexao.cursor()

***
### Criar banco de dados
***

In [4]:
# Cria um database e use ela
cursor.execute("CREATE DATABASE IF NOT EXISTS mydb")
cursor.execute("USE mydb;")

In [5]:
# Mostrar os banco de dados
cursor.execute("SHOW DATABASES")

In [6]:
for database in cursor:
    print(database)

('information_schema',)
('mydb',)
('mysql',)
('notebook_db',)
('performance_schema',)
('sys',)


***
### Criando uma tabela
***

In [7]:
# Criar uma tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS cidades (
    id INT AUTO_INCREMENT PRIMARY KEY,
    cidade VARCHAR(255),
    uf VARCHAR(2)
);
""")

In [8]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    nome VARCHAR(255) NOT NULL,
    cpf VARCHAR(11) NOT NULL,
    email VARCHAR(100) NOT NULL,
    cidade_id INT,
    criado_em DATE NOT NULL,
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
);
""")

In [9]:
# Mostrar tabelas
cursor.execute("""SHOW TABLES""")

In [10]:
for table in cursor:
    print(table)

('cidades',)
('clientes',)


***
### Inserindo registros
***

In [11]:
cidades = [('Campinas','SP'),
           ('Sao Paulo','SP'),
           ('Rio de Janeiro','RJ')]

In [12]:
cursor.executemany("""
INSERT INTO cidades (cidade, uf) VALUES (%s, %s)
""", cidades)

In [13]:
clientes = [('Regis', '01119239423', 'regis@gmail.com', 1, '2014-06-08'),
            ('Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09'),
            ('Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09'),
            ('Matheus', '01293844323', 'matheus@email.com', 3, '2014-06-08')]

In [14]:
cursor.executemany("""
INSERT INTO clientes (nome, cpf, email, cidade_id, criado_em) VALUES (%s, %s, %s, %s, %s)
""", clientes)

In [15]:
# Salvando os dados no BD.
conexao.commit()

In [16]:
# Verifica quantos inserções foram feitas
print(cursor.rowcount, "dados inseridos.")
# Pega o ID do ultimo elemento inseridp
print("ID:", cursor.lastrowid)

4 dados inseridos.
ID: 1


***
### Lendo os dados
***

In [17]:
# Registros de clientes
cursor.execute("""
SELECT * FROM clientes;
""")

In [18]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, datetime.date(2014, 6, 8))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8))


In [19]:
# Registros de cidades
cursor.execute("""
SELECT * FROM cidades;
""")

In [20]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Campinas', 'SP')
(2, 'Sao Paulo', 'SP')
(3, 'Rio de Janeiro', 'RJ')


In [21]:
# Registros de clientes e cidades
cursor.execute("""
SELECT * FROM clientes INNER JOIN cidades ON clientes.cidade_id = cidades.id;
""")

In [22]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, datetime.date(2014, 6, 8), 1, 'Campinas', 'SP')
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9), 2, 'Sao Paulo', 'SP')
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9), 2, 'Sao Paulo', 'SP')
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8), 3, 'Rio de Janeiro', 'RJ')


***
### Alterando os dados
***

In [23]:
id_cliente = 1
novo_email = 'regis_novo@gmail.com'
novo_criado_em = '2014-06-11'

In [24]:
cursor.execute("""
UPDATE clientes
SET email = %s, criado_em = %s
WHERE id = %s
""", (novo_email, novo_criado_em, id_cliente))

In [25]:
conexao.commit()

In [26]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [27]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, datetime.date(2014, 6, 11))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8))


***
### Deletando um registro
***

In [28]:
id_cliente = 4

In [29]:
cursor.execute("""
DELETE FROM clientes
WHERE id = %s
""", (id_cliente,))

In [30]:
conexao.commit()

In [31]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [32]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, datetime.date(2014, 6, 11))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))


In [33]:
# Deleta todas as instâncias
cursor.execute("DELETE FROM clientes")
cursor.execute("DELETE FROM cidades")

In [34]:
conexao.commit()

***
### Adicionar uma nova coluna
***

In [35]:
cursor.execute("""
ALTER TABLE clientes
ADD COLUMN bloqueado BOOLEAN;
""")

In [36]:
conexao.commit()

***
### Fechando conexão
***

In [37]:
cursor.execute("DROP TABLE IF EXISTS clientes")
cursor.execute("DROP TABLE IF EXISTS cidades")
cursor.execute("DROP DATABASE IF EXISTS mydb")
conexao.commit()

In [38]:
cursor.close()
conexao.close()